In [48]:
import numpy as np
import pandas as pd
import scipy.stats as stats
from scipy.stats import shapiro
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, Math, clear_output
import ipywidgets as widgets
from statsmodels.multivariate.manova import MANOVA
import pingouin as pg

print("All libraries imported successfully.")
slider = widgets.IntSlider(value=5, min=0, max=10, description='Test Slider:')
display(slider)

All libraries imported successfully.


IntSlider(value=5, description='Test Slider:', max=10)

In [74]:
# Fungsi untuk memasukkan data
def input_data():
    # Widget untuk input data
    data_input = widgets.Textarea(
        value='''3,5,2\n4,5.5,4\n2,4,3\n6,7,5\n8,10,9\n2,5,4\n5,7.5,6''',
        placeholder='Masukkan data Anda (nilai dipisahkan dengan koma, baris baru untuk setiap observasi)',
        description='Data:',
        layout=widgets.Layout(width='50%', height='150px'),
        style={'description_width': 'initial'}
    )
    # Tombol untuk mengirim data
    submit_button = widgets.Button(
        description='Kirim Data',
        button_style='success',
        tooltip='Klik untuk mengirim data',
        icon='check'
    )
    # Output widget untuk menampilkan hasil
    output = widgets.Output()

    # Tampilan input data
    display(widgets.VBox([data_input, submit_button, output]))
    
    # Fungsi untuk menangani klik tombol
    def on_submit(b):
        with output:
            clear_output()
            data_str = data_input.value.strip()
            # Memproses data input
            data = []
            for line in data_str.split('\n'):
                row = [float(x.strip()) for x in line.strip().split(',') if x.strip()]
                data.append(row)
            global df
            df = pd.DataFrame(data)
            print("## Data yang Dimasukkan:")
            display(df)
            
            # Setelah data dimasukkan, jalankan semua tes
            perform_all_tests(df)
    
    submit_button.on_click(on_submit)
    return data_input


In [76]:
# Fungsi untuk menjalankan semua tes
def perform_all_tests(df):
    perform_normality_test(df)
    perform_single_mean_test(df)


In [78]:
# Fungsi untuk Uji Kecocokan Normalitas
def perform_normality_test(df):
    print("\n\n# =======================================================")
    print("# Uji Kecocokan untuk Normalitas")
    print("# =======================================================\n")
    # Histogram dan Q-Q Plot untuk setiap variabel
    for column in df.columns:
        plt.figure(figsize=(12, 5))
        plt.subplot(1, 2, 1)
        sns.histplot(df[column], kde=True)
        plt.title(f'Histogram untuk Variabel {column}')
        plt.subplot(1, 2, 2)
        stats.probplot(df[column], dist="norm", plot=plt)
        plt.title(f'Q-Q Plot untuk Variabel {column}')
        plt.show()
        # Uji Shapiro-Wilk
        stat, p = shapiro(df[column])
        print(f"Uji Shapiro-Wilk untuk Variabel {column}: Statistik = {stat:.4f}, p-value = {p:.4f}")
        if p > 0.05:
            print(f"Kesimpulan: Data variabel {column} berdistribusi normal (gagal tolak H0).")
        else:
            print(f"Kesimpulan: Data variabel {column} tidak berdistribusi normal (tolak H0).")
    # Uji Mardia's Test untuk multivariat
    try:
        mardia = pg.multivariate_normality(df, alpha=0.05)
        print("\n### Uji Normalitas Multivariat (Mardia's Test)")
        print(f"Skewness: Statistik = {mardia[1]['stat']:.4f}, p-value = {mardia[1]['p']:.4f}")
        print(f"Kurtosis: Statistik = {mardia[2]['stat']:.4f}, p-value = {mardia[2]['p']:.4f}")
        if mardia[0]:
            print("Kesimpulan: Data berdistribusi normal multivariat (gagal tolak H0).")
        else:
            print("Kesimpulan: Data tidak berdistribusi normal multivariat (tolak H0).")
    except Exception as e:
        print(f"Terjadi kesalahan saat melakukan Mardia's Test: {e}")


In [80]:
# Fungsi untuk Uji Hipotesis Vektor Mean Populasi Tunggal
def perform_single_mean_test(df):
    print("\n\n# =======================================================")
    print("# Uji Hipotesis Vektor Mean Populasi Tunggal")
    print("# =======================================================\n")
    # Output widget untuk menampilkan hasil
    output_mean = widgets.Output()
    # Input vektor mean hipotesis
    mean_input = widgets.Text(
        value=', '.join(['0'] * df.shape[1]),
        placeholder='Masukkan nilai mean hipotesis, dipisahkan dengan koma',
        description='Mean Hipotesis:',
        style={'description_width': 'initial'}
    )
    submit_mean = widgets.Button(
        description='Kirim Mean Hipotesis',
        button_style='success',
        icon='check'
    )
    display(widgets.VBox([mean_input, submit_mean, output_mean]))
    
    def on_submit_mean(b):
        with output_mean:
            clear_output()
            hypo_mean = [float(x.strip()) for x in mean_input.value.strip().split(',')]
            if len(hypo_mean) != df.shape[1]:
                print("Panjang mean hipotesis harus sama dengan jumlah variabel.")
                return
            hypo_mean = np.array(hypo_mean)
            n = df.shape[0]
            p = df.shape[1]
            sample_mean = df.mean().values
            S = df.cov().values
            # Hitung statistik Hotelling's T-squared
            diff = sample_mean - hypo_mean
            T2 = n * diff.T @ np.linalg.inv(S) @ diff
            # Hitung nilai F
            F = (n - p) / (p * (n - 1)) * T2
            df1 = p
            df2 = n - p
            p_value = 1 - stats.f.cdf(F, df1, df2)
            # Tampilkan hasil
            display(Math(rf"\text{{Statistik Hotelling's T-squared:}} T^2 = {T2:.4f}"))
            display(Math(rf"\text{{Nilai F:}} F = {F:.4f}"))
            display(Math(rf"\text{{Derajat kebebasan:}} df_1 = {df1}, df_2 = {df2}"))
            display(Math(rf"\text{{p-value:}} {p_value:.4f}"))
            if p_value < 0.05:
                print("Kesimpulan: Tolak H0. Vektor mean populasi berbeda dengan mean hipotesis.")
            else:
                print("Kesimpulan: Gagal tolak H0. Vektor mean populasi tidak berbeda secara signifikan dengan mean hipotesis.")
            # Lanjutkan ke tes berikutnya
            perform_two_mean_test(df)
    
    submit_mean.on_click(on_submit_mean)

In [82]:
# Fungsi untuk Uji Hipotesis Vektor Mean Dua Populasi
def perform_two_mean_test(df):
    print("\n\n# =======================================================")
    print("# Uji Hipotesis Vektor Mean Dua Populasi")
    print("# =======================================================\n")
    print("Silakan masukkan data untuk Populasi Kedua.")
    # Output widget untuk menampilkan hasil
    output_two_mean = widgets.Output()
    # Input data populasi kedua
    data_input2 = widgets.Textarea(
        value='''4,6,3\n5,6.5,5\n3,5,4\n7,8,6\n9,11,10\n3,6,5\n6,8.5,7''',
        placeholder='Masukkan data Populasi Kedua (nilai dipisahkan dengan koma, baris baru untuk setiap observasi)',
        description='Data Populasi 2:',
        layout=widgets.Layout(width='50%', height='150px'),
        style={'description_width': 'initial'}
    )
    submit_button2 = widgets.Button(
        description='Kirim Data Populasi 2',
        button_style='success',
        tooltip='Klik untuk mengirim data',
        icon='check'
    )
    display(widgets.VBox([data_input2, submit_button2, output_two_mean]))
    
    def on_submit2(b):
        with output_two_mean:
            clear_output()
            data_str = data_input2.value.strip()
            data = []
            for line in data_str.split('\n'):
                row = [float(x.strip()) for x in line.strip().split(',') if x.strip()]
                data.append(row)
            df2 = pd.DataFrame(data)
            if df.shape[1] != df2.shape[1]:
                print("Jumlah variabel dalam kedua populasi harus sama.")
                return
            # Uji perbedaan mean menggunakan Hotelling's T-squared
            n1, n2 = df.shape[0], df2.shape[0]
            p = df.shape[1]
            mean1 = df.mean().values
            mean2 = df2.mean().values
            S1 = df.cov().values
            S2 = df2.cov().values
            Sp = ((n1 - 1) * S1 + (n2 - 1) * S2) / (n1 + n2 - 2)
            diff = mean1 - mean2
            T2 = (n1 * n2) / (n1 + n2) * diff.T @ np.linalg.inv(Sp) @ diff
            # Hitung nilai F
            F = (n1 + n2 - p - 1) / (p * (n1 + n2 - 2)) * T2
            df1 = p
            df2_value = n1 + n2 - p - 1
            p_value = 1 - stats.f.cdf(F, df1, df2_value)
            # Tampilkan hasil
            display(Math(rf"\text{{Statistik Hotelling's T-squared:}} T^2 = {T2:.4f}"))
            display(Math(rf"\text{{Nilai F:}} F = {F:.4f}"))
            display(Math(rf"\text{{Derajat kebebasan:}} df_1 = {df1}, df_2 = {df2_value}"))
            display(Math(rf"\text{{p-value:}} {p_value:.4f}"))
            if p_value < 0.05:
                print("Kesimpulan: Tolak H0. Vektor mean kedua populasi berbeda secara signifikan.")
            else:
                print("Kesimpulan: Gagal tolak H0. Tidak ada perbedaan signifikan antara vektor mean kedua populasi.")
            # Visualisasi
            p = df.shape[1]
            for i in range(p):
                plt.figure()
                sns.kdeplot(df.iloc[:, i], label='Populasi 1')
                sns.kdeplot(df2.iloc[:, i], label='Populasi 2')
                plt.title(f'Perbandingan Variabel {i}')
                plt.legend()
                plt.show()
            # Lanjutkan ke tes berikutnya
            perform_covariance_test(df, df2)
    
    submit_button2.on_click(on_submit2)

In [84]:
# Fungsi untuk Uji Kesamaan Matriks Kovarians Dua Populasi
def perform_covariance_test(df, df2=None):
    print("\n\n# =======================================================")
    print("# Uji Kesamaan Matriks Kovarians Dua Populasi")
    print("# =======================================================\n")
    output_covariance = widgets.Output()
    if df2 is None:
        print("Silakan masukkan data untuk Populasi Kedua.")
        # Input data populasi kedua
        data_input2 = widgets.Textarea(
            value='''4,6,3\n5,6.5,5\n3,5,4\n7,8,6\n9,11,10\n3,6,5\n6,8.5,7''',
            placeholder='Masukkan data Populasi Kedua (nilai dipisahkan dengan koma, baris baru untuk setiap observasi)',
            description='Data Populasi 2:',
            layout=widgets.Layout(width='50%', height='150px'),
            style={'description_width': 'initial'}
        )
        submit_button2 = widgets.Button(
            description='Kirim Data Populasi 2',
            button_style='success',
            tooltip='Klik untuk mengirim data',
            icon='check'
        )
        display(widgets.VBox([data_input2, submit_button2, output_covariance]))
        
        def on_submit2(b):
            with output_covariance:
                clear_output()
                data_str = data_input2.value.strip()
                data = []
                for line in data_str.split('\n'):
                    row = [float(x.strip()) for x in line.strip().split(',') if x.strip()]
                    data.append(row)
                df2_local = pd.DataFrame(data)
                if df.shape[1] != df2_local.shape[1]:
                    print("Jumlah variabel dalam kedua populasi harus sama.")
                    return
                # Lanjutkan dengan uji Box's M
                try:
                    box_m = pg.box_m(pd.concat([df, df2_local], axis=0), np.array([0]*df.shape[0] + [1]*df2_local.shape[0]))
                    print(f"Statistik Box's M: {box_m['stat'].values[0]:.4f}")
                    print(f"p-value: {box_m['p'].values[0]:.4f}")
                    if box_m['p'].values[0] < 0.05:
                        print("Kesimpulan: Tolak H0. Matriks kovarians kedua populasi berbeda secara signifikan.")
                    else:
                        print("Kesimpulan: Gagal tolak H0. Tidak ada perbedaan signifikan antara matriks kovarians kedua populasi.")
                except Exception as e:
                    print(f"Terjadi kesalahan saat melakukan Box's M Test: {e}")
        submit_button2.on_click(on_submit2)
    else:
        display(output_covariance)
        with output_covariance:
            # Jika df2 sudah diberikan
            try:
                box_m = pg.box_m(pd.concat([df, df2], axis=0), np.array([0]*df.shape[0] + [1]*df2.shape[0]))
                print(f"Statistik Box's M: {box_m['stat'].values[0]:.4f}")
                print(f"p-value: {box_m['p'].values[0]:.4f}")
                if box_m['p'].values[0] < 0.05:
                    print("Kesimpulan: Tolak H0. Matriks kovarians kedua populasi berbeda secara signifikan.")
                else:
                    print("Kesimpulan: Gagal tolak H0. Tidak ada perbedaan signifikan antara matriks kovarians kedua populasi.")
            except Exception as e:
                print(f"Terjadi kesalahan saat melakukan Box's M Test: {e}")

In [86]:
# Memulai program
print("## Uji Hipotesis dalam Metode Statistik Multivariat")
print("Silakan masukkan data Anda.")
data_input = input_data()


## Uji Hipotesis dalam Metode Statistik Multivariat
Silakan masukkan data Anda.
